In [1]:
import datetime, time
import simpy

import shapely.geometry


import pandas as pd
import openclsim.core as core
import openclsim.model as model

# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}
keep_resources = {}

In [2]:
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasContainer,
        core.HasResource,
    ),
    {},
)

TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.ContainerDependentMovable,
        core.Processor,
        core.LoadingFunction,
        core.UnloadingFunction,
        core.HasResource,
    ),
    {},
)

In [3]:
location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)  # lon, lat
location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)  # lon, lat

data_from_site = {
    "env": my_env,
    "name": "Winlocatie",
    "geometry": location_from_site,
    "capacity": 5_000_000,
    "level": 5_000_000,
}


data_to_site = {
    "env": my_env,
    "name": "Dumplocatie",
    "geometry": location_to_site,
    "capacity": 5_000_000,
    "level": 0,
}

from_site = Site(**data_from_site)
to_site = Site(**data_to_site)

In [4]:
data_hopper = {
    "env": my_env,
    "name": "Hopper 01",
    "geometry": location_from_site,
    "capacity":1000,
    "compute_v":lambda x: 10 + 2 * x,
    "loading_rate":1,
    "unloading_rate":5,
}

hopper = TransportProcessingResource(**data_hopper)

In [6]:
single_run, activity, while_activity  = model.single_run_process(
    name="single_run",
    registry={},
    env=my_env,
    origin=from_site,
    destination=to_site,
    mover=hopper,
    loader=hopper,
    unloader=hopper
)

In [7]:
my_env.run()

In [9]:
pd.DataFrame(hopper.log)

,Message,Timestamp,Value,Geometry,ActivityID,ActivityState
0,move activity single_run sailing empty of Hopp...,1970-01-01 00:00:00.000000,0.0,POINT (4.18055556 52.18664444),d2e91b93-9c5b-4298-8643-7f60f60d3218,START
1,move activity single_run sailing empty of Hopp...,1970-01-01 00:00:00.000000,0.0,POINT (4.18055556 52.18664444),d2e91b93-9c5b-4298-8643-7f60f60d3218,STOP
2,Shift amount activity single_run loading trans...,1970-01-01 00:00:00.000000,1000.0,POINT (4.18055556 52.18664444),31e5b246-a611-4ca5-a28f-f7735b9f5399,START
3,Shift amount activity single_run loading trans...,1970-01-01 00:00:00.000000,1000.0,POINT (4.18055556 52.18664444),31e5b246-a611-4ca5-a28f-f7735b9f5399,START
4,Shift amount activity single_run loading trans...,1970-01-01 00:16:40.000000,1000.0,POINT (4.18055556 52.18664444),31e5b246-a611-4ca5-a28f-f7735b9f5399,STOP
...,...,...,...,...,...,...
59995,move activity single_run sailing filled of Hop...,1970-06-19 11:03:35.928750,1000.0,POINT (4.25222222 52.11428333),3fb8b39b-0f3f-48b6-935d-e999fcade91b,STOP
59996,Shift amount activity single_run unloading tra...,1970-06-19 11:03:35.928750,1000.0,POINT (4.25222222 52.11428333),2f5b8232-f48e-4ec7-9e1f-5380ea97d355,START
59997,Shift amount activity single_run unloading tra...,1970-06-19 11:03:35.928750,1000.0,POINT (4.25222222 52.11428333),2f5b8232-f48e-4ec7-9e1f-5380ea97d355,START
59998,Shift amount activity single_run unloading tra...,1970-06-19 11:06:55.928750,1000.0,POINT (4.25222222 52.11428333),2f5b8232-f48e-4ec7-9e1f-5380ea97d355,STOP
